In [68]:
import codecs
import pandas as pd, requests, json

df = pd.read_table('surfacewaterdata', sep='\t', lineterminator='\r')

df2 = df[["site_no", "station_nm", "site_tp_cd"]]

#df3 = df2.drop([0, 133])

siteNumbers = df2["site_no"]
siteNumbers.size

440

In [119]:
# first import the functions for downloading data from NWIS
import dataretrieval.nwis as nwis
from datetime import date
import datetime
import codecs
import pandas as pd, requests, json

#create empty dataframes
newDF = pd.DataFrame()

#assign start and end dates
days = datetime.timedelta(1)
new_date = date.today() - days
newest_date = (str(new_date))

for i in range(10):
    if i==0:
        continue
    df = nwis.get_record(sites=siteNumbers[i], service='dv', start=newest_date, end=date.today())
    if (df.empty):
        continue
    a_list = df.index.tolist()
    if(len(a_list) > 0):
        a_list[0] = str(a_list[0]).replace(" 00:00:00+00:00", "")
        df.index = a_list
    newDF = newDF.append(df, ignore_index = False)
    newDF["date"] = newDF.index
    newDF[['year','month', 'day']] = newDF.date.str.split("-",expand=True)
finalDF = newDF[["00065_Mean", "00065_Maximum", "00065_Minimum", "site_no", "00065_Mean_cd", "00065_Maximum_cd", "00065_Minimum_cd", 'year','month', 'day', "date"]]
finalDF

,00065_Mean,00065_Maximum,00065_Minimum,site_no,00065_Mean_cd,00065_Maximum_cd,00065_Minimum_cd,year,month,day,date
2020-07-20,1.00,NaN,NaN,02043433,P,NaN,NaN,2020,07,20,2020-07-20
2020-07-20,6.43,6.58,6.26,0204382800,P,P,P,2020,07,20,2020-07-20


In [85]:
df = nwis.get_record(sites='02043433', service='dv', start=newest_date, end=date.today())
df.index.astype('str')
df

,00065_Mean,00065_Mean_cd,site_no
datetime,,,
2020-07-20 00:00:00+00:00,1.0,P,02043433


In [ ]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [ ]:
useful_columns = ['site_no', 'station_nm', 'site_tp_cd']
geojson_dict = df_to_geojson(groundwater, properties=useful_columns)
geojson_str = json.dumps(geojson_dict, indent=2)

In [ ]:
# save the geojson result to a file
output_filename = 'groundwater_points.geojson'
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))